# APSP(All Pairs Shortest Paths) Algorithm

> ## negative weight edge is okay
> ## but negative cycle is not okay

In [20]:
from typing import List
INF=float('inf')


def help_print(matrix:List[List]) :
    for i in range(len(matrix[0])) :
        for j in range(len(matrix[0])) :
            print(f'{matrix[i][j]}\t', end="")
        print("")
    print("")

<img src="image_APSP_image.png" width="500">

> ## Naive approach(SLOW ASAP)

In [22]:
def extend_shortest_paths(before:List[List], weight:List[List], after:List[List], num_vertex:int):
    for i in range(num_vertex) :
        for j in range(num_vertex) :
            minimum=float('inf')
            for k in range(num_vertex) :
                candidate=before[i][k]+weight[k][j]
                if candidate<minimum :
                    minimum=candidate
                    after[i][j]=minimum


def slow_asap(weight, before, after, num_vertex) :
    for i in range(num_vertex-1) :
        extend_shortest_paths(before, weight, after, num_vertex)
        before=[row[:] for row in after] #(caution) must be deep copy.
        print(f"[after ({i+1}) iteration]")
        help_print(after)
    return after

#example

weight=[    [0, 3, 8, INF],
            [INF, 0, 4, 11],
            [INF, INF, 0, 7],
            [4, INF, INF, 0]]

# initialize L^(0)
before = [[INF]*4 for _ in range(4)]
for i in range(4):
    before[i][i] = 0
after=[[INF]*4 for _ in range(4)]

print("[INITIAL Weight]")
help_print(weight)

print("[INITIAL BEFORE]")
help_print(before)

print("[INITIAL AFTER]")
help_print(after)

slow_asap(weight, before, after, 4)

[INITIAL Weight]
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	inf	inf	0	

[INITIAL BEFORE]
0	inf	inf	inf	
inf	0	inf	inf	
inf	inf	0	inf	
inf	inf	inf	0	

[INITIAL AFTER]
inf	inf	inf	inf	
inf	inf	inf	inf	
inf	inf	inf	inf	
inf	inf	inf	inf	

[after (1) iteration]
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	inf	inf	0	

[after (2) iteration]
0	3	7	14	
15	0	4	11	
11	inf	0	7	
4	7	12	0	

[after (3) iteration]
0	3	7	14	
15	0	4	11	
11	14	0	7	
4	7	11	0	



[[0, 3, 7, 14], [15, 0, 4, 11], [11, 14, 0, 7], [4, 7, 11, 0]]

> ## FASTER-APSP

In [42]:
def faster_apsp(weight:List[List], num_vertex:int) :
    L=[row[:] for row in weight] #deep copy
    r=1
    i=1
    while r<num_vertex :
        M = [[INF]*num_vertex for _ in range(num_vertex)]
        extend_shortest_paths(L, L, M, num_vertex)
        r*=2
        L = [row[:] for row in M]
        print(f'After ({i}) while loop')
        help_print(L)
        i+=1
    return L


#example

weight=[    [0, 3, 8, INF],
            [INF, 0, 4, 11],
            [INF, INF, 0, 7],
            [4, INF, INF, 0]]

print("[INITIAL Weight]")
help_print(weight)

faster_apsp(weight, 4)

[INITIAL Weight]
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	inf	inf	0	

After (1) while loop
0	3	7	14	
15	0	4	11	
11	inf	0	7	
4	7	12	0	

After (2) while loop
0	3	7	14	
15	0	4	11	
11	14	0	7	
4	7	11	0	



[[0, 3, 7, 14], [15, 0, 4, 11], [11, 14, 0, 7], [4, 7, 11, 0]]

> ## Floyd-Warshall algorithm
>> consider paths whose intermediate vertices are in {1~k}

>> ### Floyd-Warshall Vanila version

In [30]:
def floyd_warshall(weight:List[List], num_vertex:int) :
    D = [row[:] for row in weight]
    new_D = [[None]*4 for _ in range(num_vertex)] ## new_D 없이 inplace로 수정해도 무방함 D[k][k]=0이기 때문에 걱정할 필요 없음(다음 셀에서 inplace로 구현함)
    for k in range(num_vertex) :
        for i in range(num_vertex) :
            for j in range(num_vertex) :
                cur_min=D[i][j]
                if D[i][k]+D[k][j] < cur_min  :
                    cur_min = D[i][k]+D[k][j]
                new_D[i][j]=cur_min
        D = [row[:] for row in new_D] #update for next iteration
        print(f'After ({k+1}) iteration')
        help_print(D)

    return D

#example

weight=[    [0, 3, 8, INF],
            [INF, 0, 4, 11],
            [INF, INF, 0, 7],
            [4, INF, INF, 0]]

print("[INITIAL Weight]")
help_print(weight)

floyd_warshall(weight, 4)

[INITIAL Weight]
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	inf	inf	0	

After (1) iteration
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	7	12	0	

After (2) iteration
0	3	7	14	
inf	0	4	11	
inf	inf	0	7	
4	7	11	0	

After (3) iteration
0	3	7	14	
inf	0	4	11	
inf	inf	0	7	
4	7	11	0	

After (4) iteration
0	3	7	14	
15	0	4	11	
11	14	0	7	
4	7	11	0	



[[0, 3, 7, 14], [15, 0, 4, 11], [11, 14, 0, 7], [4, 7, 11, 0]]

>> ### Floyd-Warshall inplace version

In [32]:
def floyd_warshall_inplace(weight:List[List], num_vertex:int) :
    D = [row[:] for row in weight]
    for k in range(num_vertex) :
        for i in range(num_vertex) :
            for j in range(num_vertex) :
                if D[i][k]+D[k][j] < D[i][j]  :
                    D[i][j] = D[i][k]+D[k][j]
        print(f'After ({k+1}) iteration')
        help_print(D)

    return D

#example

weight=[    [0, 3, 8, INF],
            [INF, 0, 4, 11],
            [INF, INF, 0, 7],
            [4, INF, INF, 0]]

print("[INITIAL Weight]")
help_print(weight)

floyd_warshall_inplace(weight, 4)

[INITIAL Weight]
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	inf	inf	0	

After (1) iteration
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	7	12	0	

After (2) iteration
0	3	7	14	
inf	0	4	11	
inf	inf	0	7	
4	7	11	0	

After (3) iteration
0	3	7	14	
inf	0	4	11	
inf	inf	0	7	
4	7	11	0	

After (4) iteration
0	3	7	14	
15	0	4	11	
11	14	0	7	
4	7	11	0	



[[0, 3, 7, 14], [15, 0, 4, 11], [11, 14, 0, 7], [4, 7, 11, 0]]

>> ### Floyd-Warshall FINAL

In [49]:
def floyd_warshall_final(weight:List[List], num_vertex:int) :
    D = [row[:] for row in weight]
    pi=[[INF]*num_vertex for _ in range(num_vertex)] #pi[i][j] : predecessor of j on the shortest path from i to j
    for i in range(num_vertex) :
        for j in range(num_vertex) :
            if i!=j and weight[i][j]<INF :
                pi[i][j]=i
    print("[INITIAL PI]")
    help_print(pi)
    for k in range(num_vertex) :
        for i in range(num_vertex) :
            for j in range(num_vertex) :
                if D[i][k]+D[k][j] < D[i][j]  :
                    D[i][j] = D[i][k]+D[k][j]
                    pi[i][j]=pi[k][j]
        print(f'After ({k+1}) iteration')
        help_print(D)
        help_print(pi)

    return D, pi

def print_all_pairs_shortest_path(pi:List[List], start:int, end:int) :
    if start==end : print(f'({start})', end="")
    elif pi[start][end]==INF : print(f'no path')
    else :
        print_all_pairs_shortest_path(pi, start, pi[start][end])
        print(f'->({end})', end='')

#example

weight=[    [0, 3, 8, INF],
            [INF, 0, 4, 11],
            [INF, INF, 0, 7],
            [4, INF, INF, 0]]

print("[INITIAL Weight]")
help_print(weight)

D, pi = floyd_warshall_final(weight, 4)
print("[Final Shortest Distance Matrix]")
help_print(D)

print("[Final PI matrix]")
help_print(pi)

#check path
start=2
end=1
print(f"Shortest Path From ({start}) To ({end}) : ", end="")
print_all_pairs_shortest_path(pi, start, end)
print("")


[INITIAL Weight]
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	inf	inf	0	

[INITIAL PI]
inf	0	0	inf	
inf	inf	1	1	
inf	inf	inf	2	
3	inf	inf	inf	

After (1) iteration
0	3	8	inf	
inf	0	4	11	
inf	inf	0	7	
4	7	12	0	

inf	0	0	inf	
inf	inf	1	1	
inf	inf	inf	2	
3	0	0	inf	

After (2) iteration
0	3	7	14	
inf	0	4	11	
inf	inf	0	7	
4	7	11	0	

inf	0	1	1	
inf	inf	1	1	
inf	inf	inf	2	
3	0	1	inf	

After (3) iteration
0	3	7	14	
inf	0	4	11	
inf	inf	0	7	
4	7	11	0	

inf	0	1	1	
inf	inf	1	1	
inf	inf	inf	2	
3	0	1	inf	

After (4) iteration
0	3	7	14	
15	0	4	11	
11	14	0	7	
4	7	11	0	

inf	0	1	1	
3	inf	1	1	
3	0	inf	2	
3	0	1	inf	

[Final Shortest Distance Matrix]
0	3	7	14	
15	0	4	11	
11	14	0	7	
4	7	11	0	

[Final PI matrix]
inf	0	1	1	
3	inf	1	1	
3	0	inf	2	
3	0	1	inf	

Shortest Path From (2) To (1) : (2)->(3)->(0)->(1)


> # HW07-3

<img src="image_APSP_HW7.png" width="500">

In [50]:
num_vertex=5
weight=[[0,6,1,4,6],
        [INF,0,INF,INF,INF],
        [INF,INF,0,INF,1],
        [INF,2,INF,0,4],
        [INF,3,INF,INF,0]]


In [44]:
#(c) extend-shortest-paths
# initialize L^(0)
before = [[INF]*num_vertex for _ in range(num_vertex)]
for i in range(num_vertex):
    before[i][i] = 0
after=[[INF]*num_vertex for _ in range(num_vertex)]

print("[INITIAL Weight]")
help_print(weight)

print("[INITIAL BEFORE]")
help_print(before)

print("[INITIAL AFTER]")
help_print(after)

slow_asap(weight, before, after, num_vertex)

[INITIAL Weight]
0	6	1	4	6	
inf	0	inf	inf	inf	
inf	inf	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

[INITIAL BEFORE]
0	inf	inf	inf	inf	
inf	0	inf	inf	inf	
inf	inf	0	inf	inf	
inf	inf	inf	0	inf	
inf	inf	inf	inf	0	

[INITIAL AFTER]
inf	inf	inf	inf	inf	
inf	inf	inf	inf	inf	
inf	inf	inf	inf	inf	
inf	inf	inf	inf	inf	
inf	inf	inf	inf	inf	

[after (1) iteration]
0	6	1	4	6	
inf	0	inf	inf	inf	
inf	inf	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

[after (2) iteration]
0	6	1	4	2	
inf	0	inf	inf	inf	
inf	4	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

[after (3) iteration]
0	5	1	4	2	
inf	0	inf	inf	inf	
inf	4	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

[after (4) iteration]
0	5	1	4	2	
inf	0	inf	inf	inf	
inf	4	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	



[[0, 5, 1, 4, 2],
 [inf, 0, inf, inf, inf],
 [inf, 4, 0, inf, 1],
 [inf, 2, inf, 0, 4],
 [inf, 3, inf, inf, 0]]

In [45]:
#(d) extend-shortest-paths
print("[INITIAL Weight]")
help_print(weight)

faster_apsp(weight, num_vertex)

[INITIAL Weight]
0	6	1	4	6	
inf	0	inf	inf	inf	
inf	inf	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

After (1) while loop
0	6	1	4	2	
inf	0	inf	inf	inf	
inf	4	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

After (2) while loop
0	5	1	4	2	
inf	0	inf	inf	inf	
inf	4	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

After (3) while loop
0	5	1	4	2	
inf	0	inf	inf	inf	
inf	4	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	



[[0, 5, 1, 4, 2],
 [inf, 0, inf, inf, inf],
 [inf, 4, 0, inf, 1],
 [inf, 2, inf, 0, 4],
 [inf, 3, inf, inf, 0]]

In [51]:
#(d) floyd=warshall
print("[INITIAL Weight]")
help_print(weight)

D, pi = floyd_warshall_final(weight, num_vertex)
print("[Final Shortest Distance Matrix]")
help_print(D)

print("[Final PI matrix]")
help_print(pi)

#check path
start=2
end=1
print(f"Shortest Path From ({start}) To ({end}) : ", end="")
print_all_pairs_shortest_path(pi, start, end)
print("")


[INITIAL Weight]
0	6	1	4	6	
inf	0	inf	inf	inf	
inf	inf	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

[INITIAL PI]
inf	0	0	0	0	
inf	inf	inf	inf	inf	
inf	inf	inf	inf	2	
inf	3	inf	inf	3	
inf	4	inf	inf	inf	

After (1) iteration
0	6	1	4	6	
inf	0	inf	inf	inf	
inf	inf	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

inf	0	0	0	0	
inf	inf	inf	inf	inf	
inf	inf	inf	inf	2	
inf	3	inf	inf	3	
inf	4	inf	inf	inf	

After (2) iteration
0	6	1	4	6	
inf	0	inf	inf	inf	
inf	inf	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

inf	0	0	0	0	
inf	inf	inf	inf	inf	
inf	inf	inf	inf	2	
inf	3	inf	inf	3	
inf	4	inf	inf	inf	

After (3) iteration
0	6	1	4	2	
inf	0	inf	inf	inf	
inf	inf	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

inf	0	0	0	2	
inf	inf	inf	inf	inf	
inf	inf	inf	inf	2	
inf	3	inf	inf	3	
inf	4	inf	inf	inf	

After (4) iteration
0	6	1	4	2	
inf	0	inf	inf	inf	
inf	inf	0	inf	1	
inf	2	inf	0	4	
inf	3	inf	inf	0	

inf	0	0	0	2	
inf	inf	inf	inf	inf	
inf	inf	inf	inf	2	
inf	3	inf	inf	3	
inf	4	inf	inf	inf	

After (5) iteration
0	5	1	4	2	
inf	0	inf